# Лабораторная работа №1
## Выполнил: Матюхин Григорий (НПИбд-01-21)

In [1]:
import numpy as np
from numpy import random
import copy
import functools

### Задача 1.
Задача Чему равна вероятность того, что случайный треугольник, нарисованный внутри квадрата со стороной 1, является тупоугольным?

In [2]:
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """calculate angle between two vectors with the same origin"""
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.dot(v1_u, v2_u))

def is_scalene(triangle):
    for i in range(len(triangle)):
        trig = copy.deepcopy(triangle)
        origin = trig[i]
        side1 = trig[i - 1]
        side2 = trig[(i + 1) % len(triangle)]

        # Move vectors to origin
        side1 -= origin
        side2 -= origin

        angle = angle_between(side1, side2)

        if angle > np.pi/2:
            return True
    return False

iters = 100000
count = 0

for _ in range(iters):
    # Generate a triangle
    triangle = []
    for _ in range(3):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        triangle.append(np.array([x, y]))
    if is_scalene(triangle):
        count += 1
print(count / iters)


0.72396


### Задача 2.
Чему равна вероятность того, что случайный треугольник, нарисованный внутри прямоугольника, у которого одна сторона в 2 раза длиннее другой, является тупоугольным?

In [3]:
count = 0

for _ in range(iters):
    # Generate a triangle
    triangle = []
    for _ in range(3):
        x = random.uniform(0, 2)
        y = random.uniform(0, 1)
        triangle.append(np.array([x, y]))
    if is_scalene(triangle):
        count += 1
print(count / iters)

0.7972


### Задача 3.
Пусть `a`, `b` и `c` – независимые случайные величины, распределенные равномерно на `[0,1]`. Рассмотрим квадратное уравнение `a*x2+b *x+c=0`. Чему равна вероятность того, что его решения – действительные числа?

In [4]:
count = 0

for _ in range(iters):
    a = random.uniform(0,1)
    b = random.uniform(0,1)
    c = random.uniform(0,1)
    D = b**2 - 4 * a * c
    if D >= 0:
        count += 1
print(count/iters)

0.25426


### Задача 4.
В самолете 220 мест, и все билеты проданы пассажирам. Первым в самолет заходит рассеянный учёный и, не посмотрев на билет, занимает первое попавшееся место. Далее пассажиры входят по одному. Если вошедший видит, что его место свободно, он занимает свое место. Если же место занято, то вошедший занимает первое попавшееся свободное место. Найдите вероятность того, что пассажир, вошедший последним, займет место согласно своему билету.

In [5]:
count = 0

for _ in range(iters):
    passengers = 220

    seats = [x for x in range(passengers)]
    # assign tickets randomly
    tickets = [x for x in range(passengers)]
    random.shuffle(tickets)

    # scientist
    tickets.pop()
    del seats[random.randint(passengers)]

    while len(seats) != 1:
        ticket = tickets.pop()
        if ticket in seats:
            seats.remove(ticket)
        else:
            choice = random.choice(seats)
            seats.remove(choice)

    last = tickets.pop()
    if last in seats:
        count += 1
        
print(count/iters)

0.50059


### Задача 5.
Диаметр круга имеет равномерное распределение на `[0,5]`. Чему равна средняя площадь круга?

In [6]:
avg_sum = 0

for _ in range(iters):
    # take radius and not diameter
    r = random.uniform(0, 2.5)
    area = np.pi * r ** 2
    avg_sum += area

print(avg_sum/iters)

6.561095802730174


### Задача 6.
Пусть `x(1)` выбирается наугад из интервала `(0,1)`. Далее `x(2)` выбирается наугад из интервала `(x(1),1)`. Далее `x(3)` выбирается наугад из интервала `(x(2),1)` и так далее, т.е. `x(n+1)` выбирается наугад из интервала `(x(n),1)`. Чему равно среднее значение произведения `x(1)x(2)...x(n)` если `n` велико?

In [7]:
avg = 0

for _ in range(iters):
    n = 1000
    prod = 1
    l_bound = 0
    for _ in range(n):
        val = random.uniform(l_bound, 1)
        l_bound = val
        prod *= val
    avg += prod

print(avg/iters)

0.36917225127760867


### Задача 7.
В поселке 2500 жителей. Каждый из них примерно 6 раз в месяц ездит на поезде в город (т.е. с вероятностью 6/30), выбирая дни поездок по случайным мотивам независимо от остальных. Поезд ходит один раз в сутки. Какой наименьшей вместимостью должен обладать поезд, чтобы он переполнялся в среднем не чаще одного раза в 100 дней (т.е. с вероятностью 1\100)?

In [8]:
people = 2500
trip_prob = 0.2
desired_overload_ratio = 0.01
current_overload_ratio = 1
current_train_cap = 0
rng = np.random.default_rng()

while current_overload_ratio >= desired_overload_ratio:
    current_overload_ratio = sum(rng.binomial(people, trip_prob, iters) > current_train_cap)/iters
    current_train_cap += 1
print(current_train_cap)

547


### Задача 8.
В зале кинотеатра `N` мест и все `N` билетов были распроданы. Когда посетители пришли в зал, свет в зале не работал (номера мест не видны) и каждому пришлось выбирать себе место наугад. Чему равна вероятность того, что все `N` посетителей сели мимо своих мест (указанных в билете)?
Взять `N=5,50,100`.

In [9]:
def solve_for_n(N):
    
    random_cases = 0
    for _ in range(iters):
        seats = [x for x in range(N)]
        random.shuffle(seats)
        # assign tickets randomly
        tickets = [x for x in range(N)]
        random.shuffle(tickets)
        
        all_have_random_places = True
        for _ in range(N):
            ticket = tickets.pop()
            seat = seats.pop()
            if seat == ticket:
                all_have_random_places = False
                break
                
        random_cases += all_have_random_places
    print(f"With {N} seats probability for all visitors to get a random seat is {random_cases/iters}")
        
        
    

print(solve_for_n(5))
print(solve_for_n(50))
print(solve_for_n(500))

With 5 seats probability for all visitors to get a random seat is 0.3668
None
With 50 seats probability for all visitors to get a random seat is 0.3698
None
With 500 seats probability for all visitors to get a random seat is 0.36595
None


### Задача 9.
На факультете работает 6 профессоров, 6 доцентов, 10 старших преподавателей и 12 ассистентов. Из работников факультета случайным образом формируется комитет из 6 участников. Чему равна вероятность того, что в комитет войдет по крайней мере один человек каждой должности?

In [10]:
cases = 0

for _ in range(iters):
    people = 6 * [0] + 6 * [1] + 10 * [2] + 12 * [3]
    random.shuffle(people)
    committee = people[:6]
    
    all_roles = True
    for i in range(3 + 1):
        all_roles &= i in committee
    cases += all_roles

print(cases/iters)

0.38114


### Задача 10.
Чему равна вероятность того, что 4 случайно выбранные точки в данном выпуклом множестве являются вершинами вогнутого четырехугольника? В качестве выпуклого множества взять окружность.

In [11]:
def random_point_in_circle(R):
    """https://stackoverflow.com/a/50746409"""
    r = R * np.sqrt(random.uniform(0, 1))
    theta = random.uniform(0,1) * 2 * np.pi
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    return np.array([x, y])


def sort_cw(points):
    """sort points in clocwise direction"""
    
    # find center
    m = np.array([0.0, 0.0])
    for p in points:
        for j in range(2):
            m[j] += p[j] / 4
        
    def angle(a):
        # move to origin
        a1 = np.copy(a) - m
    
        # compute angle
        angle = np.rad2deg(np.arctan2(a1[0], a1[1]))
        
        # make angle span all 360 degs
        return angle if angle >= 0 else 360 + angle
    
    return sorted(points, key=angle)
    

def triangle_area(a, b, c):
    """
    This function calculates the area of a trinagle in determinant form
    
                 | x0 y0 1 |
    Area = 0.5 * | x1 y1 1 |
                 | x2 y2 1 |
                 OR
    Area = 0.5 * |x0 * (y1 - y2) + x1 * (y2 - y0) + x2 * (y0 - y1)|
                 
    from https://www.cuemath.com/geometry/area-of-triangle-in-determinant-form/
    """
    
    mat = np.array([
        [a[0], a[1], 1],
        [b[0], b[1], 1],
        [c[0], c[1], 1],
    ])
    return 0.5 * np.abs(np.linalg.det(mat))


def is_point_inside_triangle(triangle, point):
    """
    Find area of the passed triangle (v0, v1, v2) and the
    areas of the three triangles formed with the point
    (p, v1, v2), (p, v0, v2), (p, v0, v1).
    If the sum of these three is less than the first =>
    point is inside of the triangle
    """
    big_area = triangle_area(triangle[0], triangle[1], triangle[2])
    area1 = triangle_area(point, triangle[1], triangle[2])
    area2 = triangle_area(point, triangle[0], triangle[2])
    area3 = triangle_area(point, triangle[0], triangle[1])
    
    return (area1 + area2 + area3) <= big_area


count = 0 
for _ in range(iters):
    
    # get random points
    points = []
    for i in range(4):
        points.append(random_point_in_circle(1))
    # sort them so that the quad is not self-intersecting
    points = sort_cw(points)
    
    # for each triangle in quad,
    # check whether the spare point lies inside
    # if it does for at least one triangle => quad is concaved
    for i in range(4):
        triangle = [
            points[i],
            points[(i + 1) % 4],
            points[(i + 2) % 4],
        ]
        spare = points[(i + 3) % 4]
        if is_point_inside_triangle(triangle, spare):
            count += 1
            break
    
print(count/iters)

0.2041
